In [ ]:
%pip install -r requirements.txt

In [ ]:
# scrape https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added for 40,000 poems?

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging

In [4]:
with open('hi.txt', encoding='utf-8') as f:
    data = f.readlines()
bad_chars = ',& '
ndf = pd.DataFrame(columns=['Title', 'Author', 'Attributes', 'Poem'])
start = 0
for i in range(len(data)):
    data[i] = data[i][:-1]
    attr = None
    if data[i] == '<|endoftext|>':
        title_idx = data[start].find(' by ')
        title = data[start][1:title_idx-2]
        author = data[start][title_idx+4:]
        if author[-1] == ']':
            attr = author[author.find('[')+1:-1]
            author = author[:author.find('[')]
            attr = attr.replace('&', ' ').replace('   ', ',').lower()
            attr = attr.split(',')
            for j in range(len(attr)):
                attr[j] = attr[j].strip()
            while '' in attr:
                attr.remove('')
        pm = data[start+1:i]
        while '' in pm:
            pm.remove('')
        t = ''
        for j in range(len(pm)):
            t += pm[j].lower()+' NEWLINE\n '
        temp = [title, author, attr, t]
        ndf.loc[len(ndf)] = temp
        start = i+1


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oliver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
Poems = ndf.Poem.map(lambda x: nltk.tokenize.word_tokenize(x))

In [10]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for i in range(len(Poems)):
    for j in Poems[i]:
        if j in punc:
            Poems[i].remove(j)

In [12]:
def BiGram(dic, poem):
    a = poem
    b = poem[1:]
    for i in range(len(b)):
        if(a[i] in dic):
            if(b[i] in dic[a[i]]):
                dic[a[i]][b[i]] = dic[a[i]][b[i]]+1
            else:
                dic[a[i]][b[i]] = 1
        else:
            dic[a[i]]={b[i]:1}
            
    return(dic)

In [13]:
dicBi = {} # The dictionary for the BiGram model
for poem in Poems: #Feed it all the poems
    dicBi = BiGram(dicBi, poem) #Build the model!

In [14]:
def Next_word(word, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)

In [15]:
prev_word = "river"
generate_str = prev_word
for i in range(20):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str = generate_str+" "+ next_word

In [259]:
def NGram(dictionary, poem, nGram):
    Number_of_Ngrams = len(poem)-nGram+1

    for position in range(Number_of_Ngrams):
        words = [] 
        for nWord in range(nGram):
            words = words + [poem[nWord+position]]
        temp_dic = dictionary

        for nWord in range(nGram):
            current_word = words[nWord]
            last_word = nWord+1==nGram
            if(current_word in temp_dic):
                if(last_word):
                    temp_dic[current_word] = temp_dic[current_word]+1 #Increase the Ngram Count by 1
                else:    
                    temp_dic = temp_dic[current_word]
            else:
                create_dic = 0
                if(last_word):
                    create_dic = 1
                else:
                    create_dic = {words[-1]:1}
                
                for k in range(nGram-2,nWord,-1):
                    create_dic = {words[k]:create_dic}
                temp_dic[current_word] = create_dic
                break
    return(dictionary)

In [260]:
dic4 = {}
for i in Poems:
    dic4 = NGram(dic4, i,4)

In [261]:
def Next_word3(word1,word2,word3, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word1][word2][word3].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)
#Next_word3("NEWLINE","love","is",dic4)

In [264]:
prev_word1 = "elmer"
prev_word2 = "is"#"Next_word(prev_word1, dic)"
prev_word3 = "all"# Next_word2(prev_word1,prev_word2, dic2)
generate_str = prev_word1+" "+prev_word2 + " " +prev_word3
for i in range(50):
    next_word = Next_word3(prev_word1, prev_word2,prev_word3, dic4)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_word
    generate_str = generate_str+" "+ next_word
print(generate_str.replace("NEWLINE ","\n"))

KeyError: 'is'

In [ ]:
# how should i go about generating new lines

In [ ]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)